In [1]:
import os
import csv
from bs4 import BeautifulSoup
'''
In the xml document the & symbol caused a lot of trouble,
so by using the HTMl format of Bs4 for reading the document and then
parsing the xml into csv.

However, in the initial step the blog is converted into a csv file named text.csv,
which contain three columns. The first column is the name of the file 
from where it is read and the second column is the date of the blog post 
and the last column is text of the blog post on that particular day

The delimiters are not taken care at this stage. They are taken care 
during word tokenization
'''
filename = []
# post_data = []
# date_data = []
myFile = open('./text.csv','w')
wr = csv.writer(myFile,quoting = csv.QUOTE_ALL)
folder = 'BlogAuthorshipDataset'
wr.writerow(["FileName","Date","Post"])
for file_name in os.listdir(folder):
    print('Currently Working on File',file_name)
    file_data = open(folder + '/' + file_name,'r',encoding='latin-1')
    soup = BeautifulSoup(file_data,features='lxml')
    posts = soup.find_all('post')
    dates = soup.find_all('date')
    for post,date in zip(posts,dates):
        filename.append(file_name)
#         post_data.append(post.get_text())
#         date_data.append(date.get_text())
        wr.writerow([file_name , date.get_text(),post.get_text()])
print('Completed reading all')
myFile.close()

Currently Working on File 15365.female.34.indUnk.Cancer.xml
Currently Working on File 11253.male.26.Technology.Aquarius.xml
Currently Working on File 23166.female.25.indUnk.Virgo.xml
Currently Working on File 9289.male.23.Marketing.Taurus.xml
Currently Working on File 23191.female.23.Advertising.Taurus.xml
Currently Working on File 11762.female.25.Student.Aries.xml
Currently Working on File 21828.male.40.Internet.Cancer.xml
Currently Working on File 8349.male.24.Consulting.Cancer.xml
Currently Working on File 5114.male.25.indUnk.Scorpio.xml
Currently Working on File 8173.male.42.indUnk.Capricorn.xml
Currently Working on File 17944.female.39.indUnk.Sagittarius.xml
Currently Working on File 9470.male.25.Communications-Media.Aries.xml
Currently Working on File 24336.male.24.Technology.Leo.xml
Currently Working on File 7596.male.26.Internet.Scorpio.xml
Currently Working on File 23676.male.33.Technology.Scorpio.xml
Completed reading all


In [2]:
import pandas as pd
from nltk.tokenize import sent_tokenize
dataset = pd.read_csv('text.csv')
'''
The next for loop is to remove the \n \t \r and other delimiters 
which are present at the start and the end of the blog article

After this delimiters are removed we have used the apply function of 
pandas to sentence_tokenize the blog article and saved in the sentence.csv
file which has structure same as that of text.csv describe above
'''
for i in range(dataset.shape[0]):
    dataset.iloc[i].Post = dataset.iloc[i].Post.strip().lower()
temp = dataset.Post.apply(sent_tokenize)
dataset.to_csv('sentence.csv',index=False)

In [3]:
import pandas as pd
from nltk.tokenize import word_tokenize
dataset = pd.read_csv('text.csv')
'''
The next for loop is to remove the \n \t \r and other delimiters 
which are present at the start and the end of the blog article

After this delimiters are removed we have used the apply function of 
pandas to word_tokenize the blog article and saved in the sentence.csv
file which has structure same as that of text.csv describe above
'''
for i in range(dataset.shape[0]):
    dataset.iloc[i].Post = dataset.iloc[i].Post.strip().lower()
    
def punctuation_remover(tokens):
    return [word for word in tokens if word.isalpha()]

dataset.Post = dataset.Post.apply(word_tokenize)
dataset.Post = dataset.Post.apply(punctuation_remover)
dataset.to_csv('word.csv',index=False)

In [4]:
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
dataset = pd.read_csv('text.csv')
'''
The next for loop is to remove the \n \t \r and other delimiters 
which are present at the start and the end of the blog article

After this delimiters are removed we have used the apply function of 
pandas to word_tokenize the blog article and saved in the sentence.csv
file which has structure same as that of text.csv describe above
'''
def punctuation_remover(tokens):
    return [word for word in tokens if word.isalpha()]

for i in range(dataset.shape[0]):
    dataset.iloc[i].Post = dataset.iloc[i].Post.strip().lower()
dataset.Post = dataset.Post.apply(word_tokenize)
dataset.Post = dataset.Post.apply(punctuation_remover)
dataset.Post = dataset.Post.apply(Counter)
dataset.to_csv('frequency.csv',index=False)

In [5]:
import pandas as pd
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
dataset = pd.read_csv('text.csv')
'''
This code is to return only the stop words that are used on a 
particular blog on a particular day. I have returned only one 
occurance of stopword per post per date. To return all the 
occurance the function has to be modified a little as shown below

def include_stopwords(tokens):
    return [word for word in tokens if word in stop_words]

'''
def include_stopwords(tokens):
    return set([word for word in tokens if word in stop_words])

def punctuation_remover(tokens):
    return [word for word in tokens if word.isalpha()]

for i in range(dataset.shape[0]):
    dataset.iloc[i].Post = dataset.iloc[i].Post.strip().lower()

dataset.Post = dataset.Post.apply(word_tokenize)
dataset.Post = dataset.Post.apply(punctuation_remover)
dataset.Post = dataset.Post.apply(include_stopwords)
dataset.to_csv('stopwords.csv',index=False)

In [6]:
import pandas as pd
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
dataset = pd.read_csv('text.csv')
'''
This code is to return only the non-stopwords that are used on a 
particular blog on a particular day. 

'''
def avoid_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

def punctuation_remover(tokens):
    return [word for word in tokens if word.isalpha()]

for i in range(dataset.shape[0]):
    dataset.iloc[i].Post = dataset.iloc[i].Post.strip().lower()

dataset.Post = dataset.Post.apply(word_tokenize)
dataset.Post = dataset.Post.apply(punctuation_remover)
dataset.Post = dataset.Post.apply(avoid_stopwords)
dataset.to_csv('nonstopwords.csv',index=False)

In [7]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stop_words = stopwords.words('english')
porter = PorterStemmer()
dataset = pd.read_csv('text.csv')
'''
This snippet of code is to apply porter stemming on each non-stopping
word
'''
def apply_porter_stemmer(tokens):
    return [porter.stem(word) for word in tokens]

def avoid_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

def punctuation_remover(tokens):
    return [word for word in tokens if word.isalpha()]

for i in range(dataset.shape[0]):
    dataset.iloc[i].Post = dataset.iloc[i].Post.strip().lower()

dataset.Post = dataset.Post.apply(word_tokenize)
dataset.Post = dataset.Post.apply(punctuation_remover)
dataset.Post = dataset.Post.apply(avoid_stopwords)
dataset.Post = dataset.Post.apply(apply_porter_stemmer)
dataset.to_csv('stems.csv',index=False)

In [8]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 
dataset = pd.read_csv('text.csv')
'''
This snippet of code is to apply test-lemmitization on each 
non-stopping word
'''
def text_lemmetization(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

def avoid_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

def punctuation_remover(tokens):
    return [word for word in tokens if word.isalpha()]

for i in range(dataset.shape[0]):
    dataset.iloc[i].Post = dataset.iloc[i].Post.strip().lower()

dataset.Post = dataset.Post.apply(word_tokenize)
dataset.Post = dataset.Post.apply(punctuation_remover)
dataset.Post = dataset.Post.apply(avoid_stopwords)
dataset.Post = dataset.Post.apply(text_lemmetization)
dataset.to_csv('lemmas.csv',index=False)